In [169]:
!pip install transformers datasets seqeval torch accelerate


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [200]:
import json
from datasets import Dataset

# Load dataset
# with open("datasets/generated_ner_dataset.json", "r", encoding="utf-8") as f:
#     data = json.load(f)
# with open("datasets/refined_ner_dataset.json", "r", encoding="utf-8") as f:
with open("datasets/output.json", "r", encoding="utf-8") as f:
    data = json.load(f)
print(len(data))
with open("datasets/shared/temp.json", "r", encoding="utf-8") as f:
    expanded_data = json.load(f)
# Convert to Hugging Face Dataset format
for item in expanded_data:
    data.append(item)
print(len(data))
dataset = Dataset.from_list(data)

dataset = dataset.train_test_split(test_size=0.1)  # Split into train and test sets
train_dataset = dataset["train"]
test_dataset = dataset["test"]


289
349


In [201]:
# Extract unique tags
unique_tags = list(set(tag for sample in data for tag in sample["ner_tags"]))
print(unique_tags)  # Check unique tags

# Create label mapping
label2id = {label: i for i, label in enumerate(unique_tags)}
id2label = {i: label for label, i in label2id.items()}

print(label2id)  # Check label mappings


['B-DRONE', 'I-ACTION', 'B-TERRORIST', 'B-COMP_OP', 'I-QUESTION_TYPE', '0', 'B-TITLE', 'I-ROCKT', 'B-LOCATION', 'I-DRONE', 'B-ORG', 'I-TIME_FRAME', 'B-EARTHQUAKE', 'I-LOCATION', 'I-DATE', 'B-ACTION', 'B-FIRE', 'I-COUNT', 'I-TERRORIST', 'O', 'I-EARTHQUAKE', 'I-ALERT', 'B-DATE', 'B-COUNT', 'I-PERSON', 'B-PERSON', 'I-COMP_OP', 'I-FIRE', 'B-ROCKT', 'B-QUESTION_TYPE', 'B-TIME_FRAME', 'B-ALERT']
{'B-DRONE': 0, 'I-ACTION': 1, 'B-TERRORIST': 2, 'B-COMP_OP': 3, 'I-QUESTION_TYPE': 4, '0': 5, 'B-TITLE': 6, 'I-ROCKT': 7, 'B-LOCATION': 8, 'I-DRONE': 9, 'B-ORG': 10, 'I-TIME_FRAME': 11, 'B-EARTHQUAKE': 12, 'I-LOCATION': 13, 'I-DATE': 14, 'B-ACTION': 15, 'B-FIRE': 16, 'I-COUNT': 17, 'I-TERRORIST': 18, 'O': 19, 'I-EARTHQUAKE': 20, 'I-ALERT': 21, 'B-DATE': 22, 'B-COUNT': 23, 'I-PERSON': 24, 'B-PERSON': 25, 'I-COMP_OP': 26, 'I-FIRE': 27, 'B-ROCKT': 28, 'B-QUESTION_TYPE': 29, 'B-TIME_FRAME': 30, 'B-ALERT': 31}


In [192]:
from transformers import AutoTokenizer

model_checkpoint = "Davlan/xlm-roberta-base-ner-hrl"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example["tokens"], is_split_into_words=True, truncation=True)

    labels = []
    word_ids = tokenized_inputs.word_ids()  # Maps subwords to words
    ner_tags = example["ner_tags"]  # Original labels

    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)  # Ignore special tokens
        elif word_idx != previous_word_idx:
            if word_idx < len(ner_tags):
                labels.append(label2id[ner_tags[word_idx]])  # Assign full label to first subword
            else:
                labels.append(-100) 
        else:
            if word_idx < len(ner_tags):
                labels.append(label2id[ner_tags[word_idx]])  # Keep the same label
            else:
                labels.append(-100)

        previous_word_idx = word_idx

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


train_dataset = train_dataset.map(tokenize_and_align_labels)
test_dataset = test_dataset.map(tokenize_and_align_labels)

Map: 100%|██████████| 35/35 [00:00<00:00, 2691.96 examples/s]


# Checking if the mapping was done successfully:

In [194]:
sample = train_dataset[298]  # Get the first example
tokens = tokenizer.convert_ids_to_tokens(sample["input_ids"])  # Convert token IDs to words
labels = sample["labels"]  # Get corresponding labels

for token, label in zip(tokens, labels):
    print(f"{token:15} --> {id2label.get(label, 'IGNORED')}")


<s>             --> IGNORED
▁How            --> B-QUESTION_TYPE
▁many           --> I-QUESTION_TYPE
▁miss           --> B-ROCKT
ile             --> B-ROCKT
▁alert          --> I-ROCKT
s               --> I-ROCKT
▁where          --> O
▁there          --> O
▁in             --> O
▁K              --> B-LOCATION
far             --> B-LOCATION
▁Ne             --> I-LOCATION
har             --> I-LOCATION
▁Ha             --> I-LOCATION
Y               --> I-LOCATION
ar              --> I-LOCATION
den             --> I-LOCATION
▁?              --> O
</s>            --> IGNORED


In [203]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True 
)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at Davlan/xlm-roberta-base-ner-hrl and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([32]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([32, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [204]:
print("Label to ID Mapping:", label2id)
print("ID to Label Mapping:", id2label)
print("Number of Labels:", len(label2id))

Label to ID Mapping: {'B-DRONE': 0, 'I-ACTION': 1, 'B-TERRORIST': 2, 'B-COMP_OP': 3, 'I-QUESTION_TYPE': 4, '0': 5, 'B-TITLE': 6, 'I-ROCKT': 7, 'B-LOCATION': 8, 'I-DRONE': 9, 'B-ORG': 10, 'I-TIME_FRAME': 11, 'B-EARTHQUAKE': 12, 'I-LOCATION': 13, 'I-DATE': 14, 'B-ACTION': 15, 'B-FIRE': 16, 'I-COUNT': 17, 'I-TERRORIST': 18, 'O': 19, 'I-EARTHQUAKE': 20, 'I-ALERT': 21, 'B-DATE': 22, 'B-COUNT': 23, 'I-PERSON': 24, 'B-PERSON': 25, 'I-COMP_OP': 26, 'I-FIRE': 27, 'B-ROCKT': 28, 'B-QUESTION_TYPE': 29, 'B-TIME_FRAME': 30, 'B-ALERT': 31}
ID to Label Mapping: {0: 'B-DRONE', 1: 'I-ACTION', 2: 'B-TERRORIST', 3: 'B-COMP_OP', 4: 'I-QUESTION_TYPE', 5: '0', 6: 'B-TITLE', 7: 'I-ROCKT', 8: 'B-LOCATION', 9: 'I-DRONE', 10: 'B-ORG', 11: 'I-TIME_FRAME', 12: 'B-EARTHQUAKE', 13: 'I-LOCATION', 14: 'I-DATE', 15: 'B-ACTION', 16: 'B-FIRE', 17: 'I-COUNT', 18: 'I-TERRORIST', 19: 'O', 20: 'I-EARTHQUAKE', 21: 'I-ALERT', 22: 'B-DATE', 23: 'B-COUNT', 24: 'I-PERSON', 25: 'B-PERSON', 26: 'I-COMP_OP', 27: 'I-FIRE', 28: 'B-RO

In [205]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./ner_expanded_bert_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=12,  # Adjust as needed
    weight_decay=0.01,
    push_to_hub=False,
)


f:\shenkar\pagmar\venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [206]:
from transformers import Trainer, DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)


In [207]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.092183
2,No log,0.431031
3,No log,0.243734
4,No log,0.181929
5,No log,0.157488
6,No log,0.144754
7,No log,0.157285
8,No log,0.148218
9,No log,0.147831
10,No log,0.161752


TrainOutput(global_step=480, training_loss=0.34561227162679037, metrics={'train_runtime': 993.56, 'train_samples_per_second': 3.792, 'train_steps_per_second': 0.483, 'total_flos': 41498236713216.0, 'train_loss': 0.34561227162679037, 'epoch': 12.0})

In [182]:
!pip install evaluate


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [208]:
import numpy as np
import evaluate  # ✅ New library for metrics

# Load the seqeval metric for NER
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [[id2label[p] for p, l in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

trainer.compute_metrics = compute_metrics
results = trainer.evaluate()
print(results)


{'eval_loss': 0.15049368143081665, 'eval_precision': 0.967741935483871, 'eval_recall': 0.9523809523809523, 'eval_f1': 0.96, 'eval_accuracy': 0.9785575048732943, 'eval_runtime': 0.465, 'eval_samples_per_second': 75.269, 'eval_steps_per_second': 10.753, 'epoch': 12.0}


f:\shenkar\pagmar\venv\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [210]:
trainer.save_model("./ner_varied_model")
tokenizer.save_pretrained("./ner_varied_model")


('./ner_varied_model\\tokenizer_config.json',
 './ner_varied_model\\special_tokens_map.json',
 './ner_varied_model\\sentencepiece.bpe.model',
 './ner_varied_model\\added_tokens.json',
 './ner_varied_model\\tokenizer.json')

In [218]:
from transformers import pipeline

nlp_ner = pipeline("ner", model="./ner_varied_model", tokenizer="./ner_varied_model", aggregation_strategy="simple")

# text = "i would like you to present me with how much ice cream i ate in the last 5 days?"
text = "Missiles and rockets were fired at Tel Aviv yesterday."
text = "were there any aircraft alerts in Tel Aviv yesterday"
text = "hi im from Israel and I want to know what is the number of red alerts in the past month in kiryat gat ?"
# text = "is there any ice cream left in the freezer?"
# text = "תראה לי את כל התראות טילים במתן."
# text = "היו אתמול התראות חדירת כלי טיס עוין בתל אביב?"
# text = "היו גלידות בסופר בנצרת אתמול??"
result = nlp_ner(text)
print(result)
# labels = [entity["entity_group"] for entity in result]
# labels
# if not "ALERT_TYPE" in [entity["entity_group"] for entity in result]:
#     print(result)
# text = "הייתה חדירת מחבלים אתמול??"
# text = "كم عدد إنذارات الصواريخ التي كانت في تل أبيب أمس؟"


Device set to use cpu


[{'entity_group': 'LOCATION', 'score': np.float32(0.98947865), 'word': 'Israel', 'start': 10, 'end': 17}, {'entity_group': 'QUESTION_TYPE', 'score': np.float32(0.78987235), 'word': 'what', 'start': 36, 'end': 41}, {'entity_group': 'COUNT', 'score': np.float32(0.48255086), 'word': 'number', 'start': 48, 'end': 55}, {'entity_group': 'ALERT', 'score': np.float32(0.9938149), 'word': 'red alerts', 'start': 58, 'end': 69}, {'entity_group': 'TIME_FRAME', 'score': np.float32(0.9922793), 'word': 'past month', 'start': 76, 'end': 87}, {'entity_group': 'LOCATION', 'score': np.float32(0.9925471), 'word': 'kir', 'start': 90, 'end': 94}, {'entity_group': 'LOCATION', 'score': np.float32(0.7689394), 'word': 'yat gat', 'start': 94, 'end': 101}]


# We need to add more exsamples of questions in different topics to avoid overffiting model

In [73]:
from datasets import load_dataset

# Load the Yahoo Answers dataset
dataset = load_dataset("yahoo_answers_topics")

In [128]:
dataset["train"][1]
text = dataset["train"][4]["question_title"]
text = "Missiles and rockets were fired at Tel Aviv yesterday."
result = nlp_ner(text)
print(result)


KeyError: 'question_title'

In [219]:
def merge_subwords(ner_results):
    merged_entities = []
    current_entity = None

    for entity in ner_results:
        if current_entity and entity["entity_group"] == current_entity["entity_group"]:
            # Ensure words are consecutive, otherwise treat as separate
            if entity["start"] == current_entity["end"]:  
                current_entity["word"] += entity["word"]  # Merge subword
            else:
                merged_entities.append(current_entity)
                current_entity = entity.copy()  # Start a new entity

            current_entity["end"] = entity["end"]  # Update end position
            current_entity["score"] = (current_entity["score"] + entity["score"]) / 2  # Average confidence
        else:
            if current_entity:
                merged_entities.append(current_entity)
            current_entity = entity.copy()

    if current_entity:
        merged_entities.append(current_entity)

    return merged_entities

In [220]:
import json
expanded = []
for i in range(0,150):
    expanded.append(dataset["train"][i]["question_title"])
with open("datasets/expanded.json", "w", encoding="utf-8") as f:
    json.dump(expanded, f, ensure_ascii=False, indent=4)
# print()

KeyError: 'question_title'

In [84]:
with open("datasets/shared/temp.json", "r", encoding="utf-8") as f:
    data = json.load(f)
len(data)

60

In [154]:
!pip install dateparser

   ---------------------------------------- 0.0/295.7 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/295.7 kB 1.3 MB/s eta 0:00:01
   --------- ----------------------------- 71.7/295.7 kB 991.0 kB/s eta 0:00:01
   -------------------- ------------------- 153.6/295.7 kB 1.5 MB/s eta 0:00:01
   --------------------------- ------------ 204.8/295.7 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 295.7/295.7 kB 1.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [155]:
import dateparser
from datetime import datetime

def convert_time_frame(time_string):
    """
    Parses natural language time expressions and converts them to a date format.
    """
    parsed_date = dateparser.parse(time_string)
    if parsed_date:
        return parsed_date.strftime("%Y-%m-%d")  # Convert to string for query
    return time_string  # Return original if not recognized

In [165]:
print(convert_time_frame("1 of july 2024"))  # Test the function

2024-07-01


In [167]:
!pip install fuzzywuzzy


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [168]:
from fuzzywuzzy import process

def get_best_match(user_input, category):
    """
    Finds the closest entity match from the database/config.
    """
    options = ENTITY_MAPPINGS.get(category, {}).keys()
    best_match, score = process.extractOne(user_input, options)
    
    # If confidence is high, return the normalized value
    return ENTITY_MAPPINGS[category].get(best_match, user_input) if score > 80 else user_input

f:\shenkar\pagmar\venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
